In [1]:
import os
import numpy as np
import pandas as pd
from ieeg.auth import Session

from get_iEEG_data import *
from iEEG_helper_functions import *

In [2]:
SYNCHRONY_BROADBAND_DIRECTORY = "../../Data/synchrony/all/broadband"

In [3]:
def find_nan_segments(arr, min_length=100):
    nan_segments = []
    start_index = None
    nan_count = 0

    for i, value in enumerate(arr):
        if np.isnan(value):
            nan_count += 1
            if start_index is None:
                start_index = i
        else:
            if nan_count >= min_length:
                nan_segments.append((start_index, i - 1))
            start_index = None
            nan_count = 0

    # Check for the case where the array ends with a NaN segment
    if nan_count >= min_length:
        nan_segments.append((start_index, len(arr) - 1))

    return nan_segments

In [8]:
# Iterate through all files in SYNCHRONY_BROADBAND_DIRECTORY
for filename in os.listdir(SYNCHRONY_BROADBAND_DIRECTORY):
    # load only .npy files
    if filename.endswith(".npy"):
        # Filenames are in the format of HUP_{patient_id}.npy
        hup_id = filename.split("_")[1].split(".")[0]
        # Load the data
        og_data = np.load(os.path.join(SYNCHRONY_BROADBAND_DIRECTORY, filename))
        # Find NaN segments
        nan_segments = find_nan_segments(og_data)
        # If the first element of the first tuple in nan_segments is 0, then delete the first tuple
        if nan_segments[0][0] == 0:
            nan_segments = nan_segments[1:]
        if len(nan_segments) > 1:
            print(f"Filling incomplete data for HUP {hup_id}...")
            print(nan_segments)

Filling incomplete data for HUP 192...
[(23, 207), (4041, 4193), (5459, 6419)]
Filling incomplete data for HUP 184...
[(693, 880), (4384, 5099)]
Filling incomplete data for HUP 205...
[(23, 619), (5931, 6629)]
Filling incomplete data for HUP 202...
[(3364, 3587), (3827, 4117), (5720, 6419)]
Filling incomplete data for HUP 204...
[(2785, 2945), (3667, 4379)]
Filling incomplete data for HUP 182...
[(990, 1308), (6630, 8489)]
Filling incomplete data for HUP 185...
[(975, 1229), (4366, 4532), (6784, 6967), (6985, 7679)]
Filling incomplete data for HUP 186...
[(6106, 6296), (6313, 7199)]
Filling incomplete data for HUP 174...
[(4529, 4871), (7811, 8519)]
Filling incomplete data for HUP 189...
[(29, 128), (2299, 2999)]
Filling incomplete data for HUP 180...
[(2291, 2666), (5015, 5729)]
Filling incomplete data for HUP 217...
[(4059, 4211), (5797, 12659)]
Filling incomplete data for HUP 210...
[(20, 550), (4040, 8519)]
Filling incomplete data for HUP 172...
[(6728, 7046), (7191, 7889)]
Filling